In [2]:
import pandas as pd

rating_movies = pd.read_csv('data/ratings_movies.csv')

Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).

In [3]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

→ Модуль re предназначен для поиска шаблонов в тексте и встроен в язык, поэтому не нуждается в установке.
https://tproger.ru/translations/regular-expression-python/

8.1 Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.

У скольких фильмов не указан год их выпуска?

In [4]:
rating_movies['year_release'] = rating_movies['title'].apply(get_year_release)
rating_movies['year_release'].shape[0] - rating_movies['year_release'].count()


18

8.2 Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?

В качестве ответа запишите название этого фильма без указания года его выпуска.

In [5]:
rating_movies

,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0
...,...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller,2017.0
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller,2017.0
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror,2017.0
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi,2017.0


In [6]:
mask = rating_movies['year_release'] == 1999
#rating_movies[mask][rating_movies['rating'] == rating_movies[mask]['rating'].min()]
rating_movies[mask].groupby('title')['rating'].mean().sort_values().index[0]

'Bloodsport: The Dark Kumite (1999)'

8.3 Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?

Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [7]:
mask2 = rating_movies['year_release'] == 2010
rating_movies[mask2].groupby('genres')['rating'].mean().sort_values().index[0]

'Action|Sci-Fi'

8.4 Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?
 В качестве ответа запишите идентификатор этого пользователя.

In [8]:
rating_movies.groupby('userId')['genres'].nunique().sort_values().index[-1]
#rating_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False).index[0]

599

8.5 Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.

В качестве ответа укажите идентификатор этого пользователя.

Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.

In [14]:
rating_movies.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[True, False]).index[0]

53

8.6 Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.

Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [19]:
mask1 = rating_movies['year_release'] == 2018
rat = rating_movies[mask1].groupby('genres')['rating'].agg(['count', 'mean'])
rat[rat['count'] > 10].sort_values('mean').index[-1]

'Action|Adventure|Sci-Fi'

8.7 Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. 

Выберите верные варианты ответа, исходя из построенной таблицы:

In [43]:
rating_movies['date'] = pd.to_datetime(rating_movies['date'])
rating_movies['year_release'] = rating_movies['date'].dt.year
#rating_movies.groupby('')
rating_table = rating_movies.pivot_table(
    values='rating',
    index='genres',
    columns='year_release',
    fill_value=0
).round()
rating_table

year_release,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
genres,,,,,,,,,,,,,,,,,,,,,
(no genres listed),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,4.0
Action,3.0,4.0,0.0,0.0,3.0,3.0,3.0,4.0,3.0,3.0,...,0.0,4.0,4.0,3.0,2.0,0.0,3.0,4.0,3.0,3.0
Action|Adventure,3.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,...,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0
Action|Adventure|Animation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,4.0,4.0,3.0,4.0
Action|Adventure|Animation|Children,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,3.0,4.0,4.0,4.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sci-Fi|Thriller,3.0,3.0,0.0,4.0,2.0,2.0,4.0,3.0,0.0,3.0,...,4.0,2.0,2.0,4.0,4.0,0.0,4.0,4.0,4.0,4.0
Sci-Fi|Thriller|IMAX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,0.0
Thriller,4.0,4.0,4.0,4.0,3.0,3.0,4.0,3.0,3.0,3.0,...,3.0,3.0,4.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0
